In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import dill
import os

In [2]:
if os.path.getsize("Genome_Transmissivity_model_dill.pkl") > 0:
    with open("Genome_Transmissivity_model_dill.pkl", 'rb') as f:
        model = dill.load(f)
else:
    print("File is empty")

In [3]:
model.eval()

GenomeR0ValueModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (embedding): Embedding(5, 64)
  (linear): Linear(in_features=64, out_features=5, bias=True)
)

In [4]:
from collections import OrderedDict as od
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer

vocabs = od()
vocabs["A"] = int(1)
vocabs["G"] = int(2)
vocabs["T"] = int(3)
vocabs["C"] = int(4)
vocabs["U"] = int(5)
# print("Vocab is: {}".format(vocabs)) # sanity check that all of the possible values are reflected

vocab = vocab(vocabs, specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
tokenizer = get_tokenizer(vocab)

In [5]:
with open("SampleGenome.txt", "r") as f:
    genome = f.read()
genome

'AAGAAGGAGGTTATCAAAGAGGAAAAGATCAAGAACAATGACATCCGCCAAATCCTCTGTGCTGACCCCATCTACACTAGGATTGGGGCTGTCTTGGAGTCACACCAGAACTCCTTGATGAAAGAGCGAACTGAAACAGCAGTGGGGCAATGTGGGTGGTCCCCAATGCGTGGTGGGTTCACGAGAAGGATGAGGAGACTCATCGCAAAAGGCAACAAATACTTCATGGAATTTGACTGGACCAGATATGATGGCACAATTCCAACGCCACTCCTCTTGCATATCAAAAAACTCAGGTGGTCAATGATCAACGAGGTGCAAAGGAAAAAGTACCAATCACTTCATGACTGGTACTGTCATAACCTCGTCCACCGTAAAGTTGTTCTCCCTTCGGGTGAGATTACTGAACAACATAGAGGAAACCCATCTGGCCAGTTCTCCACAACAATGGATAACAACTGTGTGAACCTCTGGATCCAGGCTTTTGAGTTCGCCTATATGATTGGACCTGACAAAGAACTCTGGAAAAAATATGACACCTTGGTCTACGGTGATGATAGGCTCTCAACCACGCCAAAAATAGTGGACAACTATGAGGAGAAAGTTATAGAGATGTATAAGAACATTTTTGGTATGTGGGTTAAACCCGGTAAAGTCAAAATCTCTGAAACCCTAGTGGGTCTCTCCTTCTGTGGTTTTACAGTTGATCAAAACTTAGAGCCTATTCCAACTGCACCTGAAAAACTTATGGCTTCCCTGCTTAAACCATCTACTAAGCTACCTGATCTTGAATCACTCCATGGGAAACTCCTTTGCTATCAGCTGCTTTCCACCTTCTTAGATGAAGAACACCCTTTCAAGGGTTATGTGGAACAGTGTCTTGCTAGAACTAGCAAGCAACTCCGTGACTCTGGTCTTCCGGCCCGTTTCACTGAGGAGCAGCTTCGTCGCATTTGGAGGGGAGGACCAAAAACGTGCGATGGCTAGTGCTTCTGGCAA

In [6]:
genome = torch.tensor(np.stack([np.array(tokenizer(list(genome)))]))

In [16]:
import torch
import torch.nn as nn
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with torch.no_grad():
    output = model(genome)
    print(output)
    print(output.shape)
    # print("The predicted transmissivity is: {}".format(output.item()))
    # if output.item() > 0.5:
    #     print("The virus is predicted to be transmissible")
    # else:
    #     print("The virus is predicted to be non-transmissible")

tensor([[[ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341],
         [ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341],
         [ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341],
         ...,
         [ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341],
         [ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341],
         [ 0.1152,  0.4302,  0.0857, -0.3750,  0.4341]]])
torch.Size([1, 3510, 5])
